### Jupyter notebook to obtain dates in which there are available AERONET data and observations from Sentinel-2 or Landsat-8
### Pandas dataframes are built considering AERONET data ranging from 10:00AM to 11:00AM, AOT 500 Angstrom Exponent and image acquisition dates coincident with Landsat-8 or Sentinel-2.

###### AERONET data can be obtained from: https://aeronet.gsfc.nasa.gov/

In [1]:
import pandas as pd
import os
from datetime import datetime

# Pocess AERONET data to a Pandas dataframe
def aeronet_prepare(filename):
    dateparse = lambda x: datetime.strptime(x, "%d:%m:%Y %H:%M:%S")
    pd_name = pd.read_csv(filename, skiprows=6, na_values=['-999'], parse_dates={'times':[0,1]}, date_parser=dateparse)

    # Drop any row or column fully of NaNs and sort by the index
    pd_name = (pd_name.dropna(axis=1, how='all')
            .dropna(axis=0, how='all')
            .rename(columns={'Last_Processing_Date(dd/mm/yyyy)': 'Last_Processing_Date'})
            .sort_index())
    
    # Split times in new variable - day + hour
    pd_name["times"] = pd_name["times"].astype(str)
    pd_name["day"]= pd_name["times"].str.split(" ", n = 1, expand = True)[0].str.replace('-', '')
    pd_name["hour"]= pd_name["times"].str.split(" ", n = 1, expand = True)[1] 

    # Filter to satellite observation using hour
    sat_pass_filter =  pd_name['hour']<'11:00:00'
    af_sat_pass_filter = pd_name[sat_pass_filter]
    sat_pass_filter =  af_sat_pass_filter['hour']>'10:00:00'
    af_sat_pass_filter = af_sat_pass_filter[sat_pass_filter]

    #drop duplicate daily filtered aod values
    af_sat_pass_filter = af_sat_pass_filter.drop_duplicates(subset=['day'])
    
    print(af_sat_pass_filter.shape)

    return af_sat_pass_filter

#### To obtain AERONET data level 1.5 AOT files access de url: https://aeronet.gsfc.nasa.gov/cgi-bin/webtool_aod_v3
#### Select the desired station. The following AERONET stations were used in this study: Rio_Branco, Manaus_Embrapa, Itajubá, Alta_Floresta and Cuiabá-Miranda
#### We opted to use the temporal slice from 01/01/2016 to 31/12/2020
#### Using the "Aerosol Optical Depth (AOD) with Precipitable Water and Angstrom Parameter" produt.
#### at "1.5 Level - AOD (Cloud Screened) and Data Format: All Points".
#### For more about AERONET AOD DATA access: https://aeronet.gsfc.nasa.gov/new_web/data_description_AOD_V2.html
#### The authors would like to thank all the AERONET Principal Investigators and their staff for establishing and maintaining the 5 sites used in this investigation.

In [2]:
repository_path = '/home/marujo/user/Evaluating-the-impact-of-LaSRC-and-Sen2cor-atmospheric-correction-algorithms-on-Landsat-8-and-Sentin/'
manaus_aeronet_path = repository_path + 'aeronet_data/20160101_20191231_Manaus_EMBRAPA.lev15'
altafloresta_aeronet_path = repository_path + 'aeronet_data/20160101_20201231_Alta_Floresta.lev15'
cuiaba_aeronet_path = repository_path + 'aeronet_data/20160101_20201231_CUIABA-MIRANDA.lev15'
itajuba_aeronet_path = repository_path + 'aeronet_data/20160101_20201231_Itajuba.lev15'
riobranco_aeronet_path = repository_path + 'aeronet_data/20160101_20201231_Rio_Branco.lev15'

In [3]:
manaus = aeronet_prepare(manaus_aeronet_path)
altafloresta = aeronet_prepare(altafloresta_aeronet_path)
cuiaba = aeronet_prepare(cuiaba_aeronet_path)
itajuba = aeronet_prepare(itajuba_aeronet_path)
riobranco = aeronet_prepare(riobranco_aeronet_path)

(34, 50)
(214, 50)
(145, 50)
(213, 50)
(25, 47)


### Each station data is stored in a pandas dataframe as can be seen in:

In [5]:
manaus

,times,Day_of_Year,Day_of_Year(Fraction),AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_675nm,AOD_500nm,AOD_440nm,AOD_380nm,...,Exact_Wavelengths_of_AOD(um)_1020nm,Exact_Wavelengths_of_AOD(um)_870nm,Exact_Wavelengths_of_AOD(um)_675nm,Exact_Wavelengths_of_AOD(um)_500nm,Exact_Wavelengths_of_AOD(um)_440nm,Exact_Wavelengths_of_AOD(um)_380nm,Exact_Wavelengths_of_AOD(um)_340nm,Exact_Wavelengths_of_PW(um)_935nm,day,hour
41,2016-01-10 10:38:32,10,10.443426,0.157875,0.275534,0.331127,0.491604,0.808571,NaN,NaN,...,1.0190,0.8701,0.6747,0.5008,NaN,NaN,NaN,NaN,20160110,10:38:32
136,2016-01-13 10:39:43,13,13.444248,0.029604,0.039417,0.043349,0.049707,0.069165,0.073932,0.086361,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160113,10:39:43
204,2016-01-18 10:41:28,18,18.445463,0.036253,0.061381,0.073830,0.097659,0.143860,0.162144,0.190042,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160118,10:41:28
299,2016-01-25 10:43:30,25,25.446875,0.072739,0.124825,0.152825,0.217839,0.343915,0.408951,0.496503,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,NaN,0.9370,20160125,10:43:30
473,2016-02-10 10:45:54,41,41.448542,0.051324,0.073394,0.084049,0.104668,0.148896,0.166204,0.195885,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160210,10:45:54
517,2016-02-14 10:46:00,45,45.448611,0.123113,0.151347,0.157609,0.166487,0.184128,0.186223,0.194449,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160214,10:46:00
549,2016-02-15 10:51:41,46,46.452558,0.172600,0.201550,0.210406,0.229787,0.268333,0.278964,0.299075,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160215,10:51:41
597,2016-03-04 10:44:12,64,64.447361,0.068642,0.086716,0.091526,0.096924,0.113242,0.114702,0.123257,...,1.0190,0.8701,0.6747,0.5008,0.4394,0.3793,0.3405,0.9370,20160304,10:44:12
1358,2016-07-28 10:59:57,210,210.458299,0.043916,0.054081,0.061278,0.079612,0.118875,0.144340,0.178284,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160728,10:59:57
1415,2016-08-16 10:41:47,229,229.445683,0.046545,0.065937,0.076802,0.099287,0.141978,0.165747,0.195714,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160816,10:41:47


### Since we want to compare aeronet dates with satellite daypass information. We can extract the "day" information from it:

In [6]:
manaus_aeronet_days = set(manaus['day'])
print("Found {} days of aeronet data for Manaus.".format(len(manaus_aeronet_days)))
altafloresta_aeronet_days = set(altafloresta['day'])
print("Found {} days of aeronet data for Alta Floresta.".format(len(altafloresta_aeronet_days)))
cuiaba_aeronet_days = set(cuiaba['day'])
print("Found {} days of aeronet data for Cuiaba.".format(len(cuiaba_aeronet_days)))
itajuba_aeronet_days = set(itajuba['day'])
print("Found {} days of aeronet data for Itajuba.".format(len(itajuba_aeronet_days)))
riobranco_aeronet_days = set(riobranco['day'])
print("Found {} days of aeronet data for Rio Branco.".format(len(riobranco_aeronet_days)))

print("Example of dates: {}".format(manaus_aeronet_days))

Found 34 days of aeronet data for Manaus.
Found 214 days of aeronet data for Alta Floresta.
Found 145 days of aeronet data for Cuiaba.
Found 213 days of aeronet data for Itajuba.
Found 25 days of aeronet data for Rio Branco.
Example of dates: {'20170916', '20181026', '20160118', '20160210', '20160110', '20160215', '20161008', '20171102', '20160816', '20161026', '20161020', '20160922', '20180816', '20160911', '20160909', '20161208', '20161003', '20160818', '20160214', '20171123', '20180612', '20160304', '20160113', '20160912', '20160913', '20171021', '20180817', '20171023', '20181112', '20160125', '20171022', '20180602', '20160728', '20160819'}


## The following blocks defines functions to consults USGS for Landsat-8 images and Copernicus for Sentinel-2 images.
### Landat is consulted through landsatexplore package, which provides an interface to the EarthExplorer portal to search and download Landsat Collections scenes through a command-line interface or a Python API. Documentation avaible at: https://pypi.org/project/landsatxplore/ . The package can be installed through "pip install landsatxplore"

In [ ]:
earth_explorer_user = 'user'
earth_explorer_pass = 'password'

In [ ]:
sciHub_user = 'user'
sciHub_pass = 'password'

In [7]:
from landsatxplore.api import API

api = API(earth_explorer_user, earth_explorer_pass) # Earth explorer user and password

In [8]:
from datetime import datetime, date
from datetime import timedelta
from dateutil.rrule import rrule, MONTHLY
from dateutil.relativedelta import relativedelta

import os
import json
import requests


def search_landsat_earth_explorer(bbox: str, time: str, cloud: float = None):
    wlon, slat, elon, nlat = [float(v) for v in bbox.split(',')]
    startdate, enddate = time.split('/')

    scenes_result = api.search(
        dataset='LANDSAT_8_C1',
        bbox=(slat, wlon, nlat, elon),
        start_date=startdate,
        end_date=enddate,
        max_cloud_cover=cloud or 100,
        max_results=1000
    )

    return scenes_result


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)
    return next_month - timedelta(days=next_month.day)
    
    
def diff_month(d1, d2):
    return ((d1.year - d2.year) * 12 + d1.month - d2.month)


def create_wkt(ullon, ullat, lrlon, lrlat):
    """Create WKT representation using lat/long coordinates.
    Args:
        ullon - Upper Left longitude
        ullat - Upper Left Latitude
        lrlon - Lower Right Longitude
        lrlat - Lower Right Latitude
    """

    wkt_str = 'POLYGON (('
    wkt_str += '{} {},'.format(ullon, ullat)
    wkt_str += '{} {},'.format(lrlon, ullat)
    wkt_str += '{} {},'.format(lrlon, lrlat)
    wkt_str += '{} {},'.format(ullon, lrlat)
    wkt_str += '{} {}'.format(ullon, ullat)
    wkt_str += '))'

    return wkt_str


def request_copernicus(bbox, start_date, end_date):
    """Retrieve Sentinel Images from Copernicus."""
    wlon, slat, elon, nlat = bbox.split(',')
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    scenes = {}
    totres = 1000000
    first = 0
    pquery = 'https://scihub.copernicus.eu/dhus/search?format=json'
    pquery = 'https://scihub.copernicus.eu/apihub/search?format=json'
    pquery += '&q=platformname:Sentinel-2'
    if end_date is None:
        end_date = datetime.now().strftime("%Y-%m-%d")
    pquery += ' AND beginposition:[{}T00:00:00:999Z TO {}T23:59:00:999Z]'.format(start_date, end_date)
    pquery += ' AND cloudcoverpercentage:[0 TO {}]'.format(100)
    if wlon == elon and slat == nlat:
        footprintPoly = create_wkt(wlon-0.01,nlat+0.01,elon+0.01,slat-0.01)
        pquery += ' AND (footprint:"Contains({})")'.format(footprintPoly)
    else:
        footprintPoly = create_wkt(wlon,nlat,elon,slat)
        pquery += ' AND (footprint:"Intersects({})")'.format(footprintPoly)

    limit = int(100000)
    rows = min(100,limit)
    count_results = 0

    while count_results < min(limit,totres) and totres != 0:
        rows = min(100,limit-len(scenes),totres)
        first = count_results
        query = pquery + '&rows={}&start={}'.format(rows,first)
        try:
            # Using sentinel user and release on out of scope
            r = requests.get(query, auth=(sciHub_user, sciHub_pass), verify=True)

            if not r.status_code // 100 == 2:
                raise Exception('openSearchS2SAFE API returned unexpected response {}:'.format(r.status_code))
                return {}
            r_dict = r.json()

        except requests.exceptions.RequestException as exc:
            return {}

        if 'entry' in r_dict['feed']:
            totres = int(r_dict['feed']['opensearch:totalResults'])
            results = r_dict['feed']['entry']
            if not isinstance(results, list):
                results = [results]
            for result in results:
                count_results += 1
                identifier = result['title']
                type = identifier.split('_')[1]
                ### Jump level 2 images (will download and process only L1C)
                if type == 'MSIL2A':
                    continue
                scenes[identifier] = {}
                scenes[identifier]['pathrow'] = identifier.split('_')[-2][1:]
                scenes[identifier]['sceneid'] = identifier
                scenes[identifier]['type'] = identifier.split('_')[1]
                for data in result['date']:
                    if str(data['name']) == 'beginposition':
                        scenes[identifier]['date'] = str(data['content'])[0:10]
                if not isinstance(result['double'], list):
                    result['double'] = [result['double']]
                for data in result['double']:
                    if str(data['name']) == 'cloudcoverpercentage':
                        scenes[identifier]['cloud'] = float(data['content'])
                for data in result['str']:
                    if str(data['name']) == 'size':
                        scenes[identifier]['size'] = data['content']
                    if str(data['name']) == 'footprint':
                        scenes[identifier]['footprint'] = data['content']
                    if str(data['name']) == 'tileid':
                        scenes[identifier]['tileid'] = data['content']
                scenes[identifier]['link'] = result['link'][0]['href']
                scenes[identifier]['icon'] = result['link'][2]['href']
        else:
            totres = 0
    return list(scenes)


def consult_s2(bbox_list, start_date, end_date):
    time = '{}T00:00:00/{}T23:59:00'.format(
    start_date.strftime("%Y-%m-%d"), 
    end_date.strftime("%Y-%m-%d"))

    result = {}

    for bbox in bbox_list:
        print('Consulting bbox: {} for Sentinel 2 ...'.format(bbox), flush=True)
        features_external = []

        for i in range(diff_month(end_date, start_date)+1):
            current_date = start_date+relativedelta(months=+i)
            current_date_last_day = last_day_of_month(current_date)
            features_external += request_copernicus(bbox, current_date, current_date_last_day)

        list_ids_s2 = {f for f in features_external}

        return list_ids_s2


def consult_lc8(bbox_list, start_date, end_date):
    time = '{}T00:00:00/{}T23:59:00'.format(
    start_date.strftime("%Y-%m-%d"), 
    end_date.strftime("%Y-%m-%d"))
    
    result = {}

    for bbox in bbox_list:
        print('Consulting bbox: {} for Landsat-8 ...'.format(bbox), flush=True)
        features_external = []
        features_external = search_landsat_earth_explorer(bbox, time)
        list_ids_lc8 = {f['displayId'] for f in features_external}
        
        return list_ids_lc8
    
def get_s2_days(s2_sceneids):
    days = set()
    for sceneid in s2_sceneids:
        ymd = sceneid.split('_')[2][0:8]
        if ymd not in days:
            days.add(ymd) 
    return days

def s2_subset_sceneids_by_days(s2_sceneids, days):
    result=[]
    for sceneid in s2_sceneids:
        ymd = sceneid.split('_')[2][0:8]
        if ymd in days:
            result.append(sceneid) 
    return result
    
def get_lc8_days(lc8_sceneids):
    days = set()
    for sceneid in lc8_sceneids:
        ymd = sceneid.split('_')[3][0:8]
        if ymd not in days:
            days.add(ymd)
    return days

def lc8_subset_sceneids_by_days(lc8_sceneids, days):
    result=[]
    for sceneid in lc8_sceneids:
        ymd = sceneid.split('_')[3][0:8]
        if ymd in days:
            result.append(sceneid) 
    return result

### Creating the Sentinel-2 and Lansat-8 day acquisition to bounding box using function bbox_s2_lc8_consult

In [9]:
start_date = date(2016, 1, 1)
end_date = date(2020, 6, 30)

bbox_list_manaus = ['-60.15, -3.59, -59.73, -2.85']
bbox_list_altafloresta = ['-56.85, -9.9, -55.2, -9.2']
bbox_list_itajuba = ['-45.83, -22.55, -45.04, -21.8']
bbox_list_cuiaba = ['-56.8, -16.25, -55.2, -15.5']
bbox_list_riobranco = ['-67.95, -9.9, -67.25, -9.2']

### Consulting Landsat-8 dates:

In [10]:
manaus_lc8 = consult_lc8(bbox_list_manaus, start_date, end_date)
print("Manaus presented {0} Landsat-8 sceneids".format(len(manaus_lc8)))

altafloresta_lc8 = consult_lc8(bbox_list_altafloresta, start_date, end_date)
print("Alta Floresta presented {0} Landsat-8 sceneids".format(len(altafloresta_lc8)))

itajuba_lc8 = consult_lc8(bbox_list_itajuba, start_date, end_date)
print("Itajuba presented {0} Landsat-8 sceneids".format(len(itajuba_lc8)))

cuiaba_lc8 = consult_lc8(bbox_list_cuiaba, start_date, end_date)
print("Cuiaba presented {0} Landsat-8 sceneids".format(len(cuiaba_lc8)))

riobranco_lc8 = consult_lc8(bbox_list_riobranco, start_date, end_date)
print("Rio Branco presented {0} Landsat-8 sceneids".format(len(riobranco_lc8)))

Consulting bbox: -60.15, -3.59, -59.73, -2.85 for Landsat-8 ...
Manaus presented 406 Landsat-8 sceneids
Consulting bbox: -56.85, -9.9, -55.2, -9.2 for Landsat-8 ...
Alta Floresta presented 503 Landsat-8 sceneids
Consulting bbox: -45.83, -22.55, -45.04, -21.8 for Landsat-8 ...
Itajuba presented 408 Landsat-8 sceneids
Consulting bbox: -56.8, -16.25, -55.2, -15.5 for Landsat-8 ...
Cuiaba presented 308 Landsat-8 sceneids
Consulting bbox: -67.95, -9.9, -67.25, -9.2 for Landsat-8 ...
Rio Branco presented 410 Landsat-8 sceneids


### Consulting Sentinel-2 dates:

In [ ]:
manaus_s2 = consult_s2(bbox_list_manaus, start_date, end_date)
print("Manaus presented {0} Sentinel-2 sceneids".format(len(manaus_s2)))

altafloresta_s2 = consult_s2(bbox_list_altafloresta, start_date, end_date)
print("Alta Floresta presented {0} Sentinel-2 sceneids".format(len(altafloresta_s2)))

itajuba_s2 = consult_s2(bbox_list_itajuba, start_date, end_date)
print("Itajuba presented {0} Sentinel-2 sceneids".format(len(itajuba_s2)))

cuiaba_s2 = consult_s2(bbox_list_cuiaba, start_date, end_date)
print("Cuiaba presented {0} Sentinel-2 sceneids".format(len(cuiaba_s2)))

riobranco_s2 = consult_s2(bbox_list_riobranco, start_date, end_date)
print("Rio Branco presented {0} Sentinel-2 sceneids".format(len(riobranco_s2)))

Consulting bbox: -60.15, -3.59, -59.73, -2.85 for Sentinel 2 ...
Manaus presented 536 Sentinel-2 sceneids
Consulting bbox: -56.85, -9.9, -55.2, -9.2 for Sentinel 2 ...
Alta Floresta presented 796 Sentinel-2 sceneids
Consulting bbox: -45.83, -22.55, -45.04, -21.8 for Sentinel 2 ...
Itajuba presented 273 Sentinel-2 sceneids
Consulting bbox: -56.8, -16.25, -55.2, -15.5 for Sentinel 2 ...
Cuiaba presented 690 Sentinel-2 sceneids
Consulting bbox: -67.95, -9.9, -67.25, -9.2 for Sentinel 2 ...


### Now we intersect the dates containing AERONET data with the dates in which we obtained orbital observations (from Landsat-8 or Sentinel-2):

In [12]:
manaus_aeronet_lc8 = manaus_aeronet_days & get_lc8_days(manaus_lc8)
print("Manaus presented {0} dates containing both Landsat-8 and AERONET data".format(len(manaus_aeronet_lc8)))
print(manaus_aeronet_lc8) #example of dates

altafloresta_aeronet_lc8 = altafloresta_aeronet_days & get_lc8_days(altafloresta_lc8)
print("Alta Floresta presented {0} dates containing both Landsat-8 and AERONET data".format(len(altafloresta_lc8)))

cuiaba_aeronet_lc8 = cuiaba_aeronet_days & get_lc8_days(cuiaba_lc8)
print("Cuiaba presented {0} dates containing both Landsat-8 and AERONET data".format(len(cuiaba_aeronet_lc8)))

itajuba_aeronet_lc8 = itajuba_aeronet_days & get_lc8_days(itajuba_lc8)
print("Itajuba presented {0} dates containing both Landsat-8 and AERONET data".format(len(itajuba_lc8)))

riobranco_aeronet_lc8 = riobranco_aeronet_days & get_lc8_days(riobranco_lc8)
print("Rio Branco presented {0} dates containing both Landsat-8 and AERONET data".format(len(riobranco_lc8)))

Manaus presented 5 dates containing both Landsat-8 and AERONET data
{'20160110', '20160913', '20161008', '20160922', '20170916'}
Alta Floresta presented 606 dates containing both Landsat-8 and AERONET data
Cuiaba presented 22 dates containing both Landsat-8 and AERONET data
Itajuba presented 614 dates containing both Landsat-8 and AERONET data
Rio Branco presented 513 dates containing both Landsat-8 and AERONET data


In [13]:
manaus_aeronet_s2 = manaus_aeronet_days & get_s2_days(manaus_s2)
print("Manaus presented {0} dates containing both Sentinel-2 and AERONET data".format(len(manaus_aeronet_s2)))
print(manaus_aeronet_s2) #example of dates

altafloresta_aeronet_s2 = altafloresta_aeronet_days & get_s2_days(altafloresta_s2)
print("Alta Floresta presented {0} dates containing both Sentinel-2 and AERONET data".format(len(altafloresta_s2)))

cuiaba_aeronet_s2 = cuiaba_aeronet_days & get_s2_days(cuiaba_s2)
print("Cuiaba presented {0} dates containing both Sentinel-2 and AERONET data".format(len(cuiaba_s2)))

itajuba_aeronet_s2 = itajuba_aeronet_days & get_s2_days(itajuba_s2)
print("Itajuba presented {0} dates containing both Sentinel-2 and AERONET data".format(len(itajuba_s2)))

riobranco_aeronet_s2 = riobranco_aeronet_days & get_s2_days(riobranco_s2)
print("Rio Branco presented {0} dates containing both Sentinel-2 and AERONET data".format(len(riobranco_s2)))

Manaus presented 8 dates containing both Sentinel-2 and AERONET data
{'20180816', '20161008', '20171023', '20180612', '20171102', '20180602', '20160819', '20160816'}
Alta Floresta presented 796 dates containing both Sentinel-2 and AERONET data
Cuiaba presented 690 dates containing both Sentinel-2 and AERONET data
Itajuba presented 273 dates containing both Sentinel-2 and AERONET data
Rio Branco presented 235 dates containing both Sentinel-2 and AERONET data


### Exporting dataframes to csv in local disk

In [14]:
manaus_lc8_pd = manaus[manaus['day'].isin(manaus_aeronet_lc8)]
manaus_lc8_pd.to_csv(r'dataframes/manaus_lc8_pd.csv', index = False)

altafloresta_lc8_pd = altafloresta[altafloresta['day'].isin(altafloresta_aeronet_lc8)]
altafloresta_lc8_pd.to_csv(r'dataframes/altafloresta_lc8_pd.csv', index = False)

cuiaba_lc8_pd = cuiaba[cuiaba['day'].isin(cuiaba_aeronet_lc8)]
cuiaba_lc8_pd.to_csv(r'dataframes/cuiaba_lc8_pd.csv', index = False)

itajuba_lc8_pd = itajuba[itajuba['day'].isin(itajuba_aeronet_lc8)]
itajuba_lc8_pd.to_csv(r'dataframes/itajuba_lc8_pd.csv', index = False)

riobranco_lc8_pd = riobranco[riobranco['day'].isin(riobranco_aeronet_lc8)]
riobranco_lc8_pd.to_csv(r'dataframes/riobranco_lc8_pd.csv', index = False)

manaus_lc8_pd # view one of the dataframes

,times,Day_of_Year,Day_of_Year(Fraction),AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_675nm,AOD_500nm,AOD_440nm,AOD_380nm,...,Exact_Wavelengths_of_AOD(um)_1020nm,Exact_Wavelengths_of_AOD(um)_870nm,Exact_Wavelengths_of_AOD(um)_675nm,Exact_Wavelengths_of_AOD(um)_500nm,Exact_Wavelengths_of_AOD(um)_440nm,Exact_Wavelengths_of_AOD(um)_380nm,Exact_Wavelengths_of_AOD(um)_340nm,Exact_Wavelengths_of_PW(um)_935nm,day,hour
41,2016-01-10 10:38:32,10,10.443426,0.157875,0.275534,0.331127,0.491604,0.808571,NaN,NaN,...,1.0190,0.8701,0.6747,0.5008,NaN,NaN,NaN,NaN,20160110,10:38:32
1658,2016-09-13 10:30:37,257,257.437928,0.046712,0.064589,0.070753,0.083926,0.113694,0.131556,0.157846,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160913,10:30:37
1754,2016-09-22 10:55:14,266,266.455023,0.064767,0.101347,0.120950,0.168791,0.259382,0.312689,0.379435,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160922,10:55:14
1929,2016-10-08 10:38:45,282,282.443576,0.056553,0.076688,0.088229,0.117906,0.176579,0.210890,0.255529,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20161008,10:38:45
2781,2017-09-16 10:57:58,259,259.456921,0.115815,0.250429,0.332410,0.521197,0.854487,1.024291,1.217633,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,NaN,0.9353,20170916,10:57:58


In [15]:
manaus_s2_pd = manaus[manaus['day'].isin(manaus_aeronet_s2)]
manaus_s2_pd.to_csv(r'dataframes/manaus_s2_pd.csv', index = False)

altafloresta_s2_pd = altafloresta[altafloresta['day'].isin(altafloresta_aeronet_s2)]
altafloresta_s2_pd.to_csv(r'dataframes/altafloresta_s2_pd.csv', index = False)

cuiaba_s2_pd = cuiaba[cuiaba['day'].isin(cuiaba_aeronet_s2)]
cuiaba_s2_pd.to_csv(r'dataframes/cuiaba_s2_pd.csv', index = False)

itajuba_s2_pd = itajuba[itajuba['day'].isin(itajuba_aeronet_s2)]
itajuba_s2_pd.to_csv(r'dataframes/itajuba_s2_pd.csv', index = False)

riobranco_s2_pd = riobranco[riobranco['day'].isin(riobranco_aeronet_s2)]
riobranco_s2_pd.to_csv(r'dataframes/riobranco_s2_pd.csv', index = False)

manaus_s2_pd # View one of the dataframes

,times,Day_of_Year,Day_of_Year(Fraction),AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_675nm,AOD_500nm,AOD_440nm,AOD_380nm,...,Exact_Wavelengths_of_AOD(um)_1020nm,Exact_Wavelengths_of_AOD(um)_870nm,Exact_Wavelengths_of_AOD(um)_675nm,Exact_Wavelengths_of_AOD(um)_500nm,Exact_Wavelengths_of_AOD(um)_440nm,Exact_Wavelengths_of_AOD(um)_380nm,Exact_Wavelengths_of_AOD(um)_340nm,Exact_Wavelengths_of_PW(um)_935nm,day,hour
1415,2016-08-16 10:41:47,229,229.445683,0.046545,0.065937,0.076802,0.099287,0.141978,0.165747,0.195714,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160816,10:41:47
1490,2016-08-19 10:46:39,232,232.449062,0.060562,0.084561,0.098345,0.130861,0.190992,0.224858,0.266599,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20160819,10:46:39
1929,2016-10-08 10:38:45,282,282.443576,0.056553,0.076688,0.088229,0.117906,0.176579,0.210890,0.255529,...,1.0212,0.8715,0.6755,0.4990,0.4387,0.3799,0.3402,0.9353,20161008,10:38:45
3019,2017-10-23 10:16:06,296,296.427847,0.096951,0.138959,0.161447,0.218909,0.342775,0.412178,0.505781,...,1.0193,0.8700,0.6751,0.5008,0.4395,0.3800,NaN,0.9373,20171023,10:16:06
3257,2017-11-02 10:40:54,306,306.445069,0.058466,0.086564,0.101461,0.138840,0.220956,0.262301,0.322227,...,1.0193,0.8700,0.6751,0.5008,0.4395,0.3800,0.3406,0.9373,20171102,10:40:54
3962,2018-06-02 10:53:41,153,153.453947,NaN,0.063374,0.067794,0.077037,0.104015,0.115673,0.135861,...,1.0196,0.8700,0.6750,0.5014,0.4411,0.3801,0.3397,0.9362,20180602,10:53:41
4065,2018-06-12 10:51:30,163,163.452431,NaN,0.056603,0.061944,0.073981,0.103509,0.117350,0.138578,...,1.0196,0.8700,0.6750,0.5014,0.4411,0.3801,0.3397,0.9362,20180612,10:51:30
4288,2018-08-16 10:44:55,228,228.447859,NaN,0.063802,0.070652,0.087409,0.131525,0.150533,0.178675,...,1.0196,0.8700,0.6750,0.5014,0.4411,0.3801,0.3397,0.9362,20180816,10:44:55


### Sceneids of the images that will be used:

In [16]:
manaus_sceneids_lc8 = lc8_subset_sceneids_by_days(manaus_lc8, manaus_aeronet_lc8)
print("Manaus presented {0} Landsat-8 sceneids to download".format(len(manaus_sceneids_lc8)))
altafloresta_sceneids_lc8 = lc8_subset_sceneids_by_days(altafloresta_lc8, altafloresta_aeronet_lc8)
print("Alta Floresta presented {0} Landsat-8 sceneids to download".format(len(altafloresta_sceneids_lc8)))
cuiaba_sceneids_lc8 = lc8_subset_sceneids_by_days(cuiaba_lc8, cuiaba_aeronet_lc8)
print("Cuiaba presented {0} Landsat-8 sceneids to download".format(len(cuiaba_sceneids_lc8)))
itajuba_sceneids_lc8 = lc8_subset_sceneids_by_days(itajuba_lc8, itajuba_aeronet_lc8)
print("Itajuba presented {0} Landsat-8 sceneids to download".format(len(itajuba_sceneids_lc8)))
riobranco_sceneids_lc8 = lc8_subset_sceneids_by_days(riobranco_lc8, riobranco_aeronet_lc8)
print("Rio Branco presented {0} Landsat-8 sceneids to download".format(len(riobranco_sceneids_lc8)))

Manaus presented 10 Landsat-8 sceneids to download
Alta Floresta presented 83 Landsat-8 sceneids to download
Cuiaba presented 56 Landsat-8 sceneids to download
Itajuba presented 78 Landsat-8 sceneids to download
Rio Branco presented 7 Landsat-8 sceneids to download


In [17]:
manaus_sceneids_s2 = s2_subset_sceneids_by_days(manaus_s2, manaus_aeronet_s2)
print("Manaus presented {0} Sentinel-2 sceneids to download".format(len(manaus_sceneids_s2)))
altafloresta_sceneids_s2 = s2_subset_sceneids_by_days(altafloresta_s2, altafloresta_aeronet_s2)
print("Alta Floresta presented {0} Sentinel-2 sceneids to download".format(len(altafloresta_sceneids_s2)))
cuiaba_sceneids_s2 = s2_subset_sceneids_by_days(cuiaba_s2, cuiaba_aeronet_s2)
print("Cuiaba presented {0} Sentinel-2 sceneids to download".format(len(cuiaba_sceneids_s2)))
itajuba_sceneids_s2 = s2_subset_sceneids_by_days(itajuba_s2, itajuba_aeronet_s2)
print("Itajuba presented {0} Sentinel-2 sceneids to download".format(len(itajuba_sceneids_s2)))
riobranco_sceneids_s2 = s2_subset_sceneids_by_days(riobranco_s2, riobranco_aeronet_s2)
print("Rio Branco presented {0} Sentinel-2 sceneids to download".format(len(riobranco_sceneids_s2)))

Manaus presented 8 Sentinel-2 sceneids to download
Alta Floresta presented 104 Sentinel-2 sceneids to download
Cuiaba presented 68 Sentinel-2 sceneids to download
Itajuba presented 38 Sentinel-2 sceneids to download
Rio Branco presented 4 Sentinel-2 sceneids to download


In [20]:
print(manaus_sceneids_lc8)
print("")
print(manaus_sceneids_s2)

['LC08_L1TP_230063_20160922_20170321_01_T1', 'LC08_L1TP_231062_20170916_20170929_01_T1', 'LC08_L1TP_230062_20161008_20170320_01_T1', 'LC08_L1TP_231062_20160913_20170321_01_T1', 'LC08_L1TP_231063_20160913_20170321_01_T1', 'LC08_L1TP_230062_20160110_20170405_01_T1', 'LC08_L1TP_230062_20160922_20170321_01_T1', 'LC08_L1TP_230063_20161008_20170320_01_T1', 'LC08_L1TP_231063_20170916_20170929_01_T1', 'LC08_L1TP_230063_20160110_20170405_01_T1']

['S2A_MSIL1C_20180816T142041_N0206_R010_T20MRB_20180816T180323', 'S2B_MSIL1C_20180602T142039_N0206_R010_T20MRB_20180602T174131', 'S2A_MSIL1C_20171023T143321_N0206_R053_T20MRB_20171023T160302', 'S2A_MSIL1C_20161008T143312_N0204_R053_T20MRB_20161008T143313', 'S2B_MSIL1C_20180612T142039_N0206_R010_T20MRB_20180612T160557', 'S2A_MSIL1C_20160816T142322_N0204_R010_T20MRB_20160816T142325', 'S2A_MSIL1C_20160819T142752_N0204_R053_T20MRB_20160819T142755', 'S2A_MSIL1C_20171102T142751_N0206_R053_T20MRB_20171102T161043']


In [21]:
import csv

outpath = repository_path + '/sceneids/'

In [22]:
print(manaus_sceneids_lc8)
with open('{}manaus_sceneids_lc8.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(manaus_sceneids_lc8)

print(altafloresta_sceneids_lc8)
with open('{}altafloresta_sceneids_lc8.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(altafloresta_sceneids_lc8)
    
print(cuiaba_sceneids_lc8)
with open('{}cuiaba_sceneids_lc8.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(cuiaba_sceneids_lc8)
    
print(itajuba_sceneids_lc8)
with open('{}itajuba_sceneids_lc8.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(itajuba_sceneids_lc8)
    
print(riobranco_sceneids_lc8)
with open('{}riobranco_sceneids_lc8.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(riobranco_sceneids_lc8)

['LC08_L1TP_230063_20160922_20170321_01_T1', 'LC08_L1TP_231062_20170916_20170929_01_T1', 'LC08_L1TP_230062_20161008_20170320_01_T1', 'LC08_L1TP_231062_20160913_20170321_01_T1', 'LC08_L1TP_231063_20160913_20170321_01_T1', 'LC08_L1TP_230062_20160110_20170405_01_T1', 'LC08_L1TP_230062_20160922_20170321_01_T1', 'LC08_L1TP_230063_20161008_20170320_01_T1', 'LC08_L1TP_231063_20170916_20170929_01_T1', 'LC08_L1TP_230063_20160110_20170405_01_T1']
['LC08_L1GT_074067_20171022_20171107_01_T2', 'LC08_L1GT_074067_20180822_20180829_01_T2', 'LC08_L1TP_227066_20190708_20190719_01_T1', 'LC08_L1TP_228066_20190917_20190926_01_T1', 'LC08_L1TP_227067_20180603_20180615_01_T1', 'LC08_L1GT_074067_20160528_20170324_01_T2', 'LC08_L1TP_228067_20160417_20170326_01_T1', 'LC08_L1TP_228066_20160604_20170324_01_T1', 'LC08_L1TP_227066_20171022_20171107_01_T1', 'LC08_L1GT_074067_20180806_20180815_01_T2', 'LC08_L1TP_226067_20160910_20170321_01_T1', 'LC08_L1TP_228067_20160519_20170324_01_T1', 'LC08_L1TP_227067_20170920_201

In [23]:
print(manaus_sceneids_s2)
with open('{}manaus_sceneids_s2.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(manaus_sceneids_s2)

print(altafloresta_sceneids_s2)
with open('{}altafloresta_sceneids_s2.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(altafloresta_sceneids_s2)
    
print(cuiaba_sceneids_s2)
with open('{}cuiaba_sceneids_s2.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(cuiaba_sceneids_s2)
    
print(itajuba_sceneids_s2)
with open('{}itajuba_sceneids_s2.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(itajuba_sceneids_s2)
    
print(riobranco_sceneids_s2)
with open('{}riobranco_sceneids_s2.csv'.format(outpath), 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(riobranco_sceneids_s2)

['S2A_MSIL1C_20180816T142041_N0206_R010_T20MRB_20180816T180323', 'S2B_MSIL1C_20180602T142039_N0206_R010_T20MRB_20180602T174131', 'S2A_MSIL1C_20171023T143321_N0206_R053_T20MRB_20171023T160302', 'S2A_MSIL1C_20161008T143312_N0204_R053_T20MRB_20161008T143313', 'S2B_MSIL1C_20180612T142039_N0206_R010_T20MRB_20180612T160557', 'S2A_MSIL1C_20160816T142322_N0204_R010_T20MRB_20160816T142325', 'S2A_MSIL1C_20160819T142752_N0204_R053_T20MRB_20160819T142755', 'S2A_MSIL1C_20171102T142751_N0206_R053_T20MRB_20171102T161043']
['S2B_MSIL1C_20170929T140409_N0205_R067_T21LXK_20170929T140408', 'S2B_MSIL1C_20171029T140039_N0206_R067_T21LWK_20171029T171908', 'S2B_MSIL1C_20190919T140049_N0208_R067_T21LXK_20190919T153847', 'S2A_MSIL1C_20160601T140102_N0202_R067_T21LXK_20160601T140057', 'S2A_MSIL1C_20180601T140051_N0206_R067_T21LXK_20180601T153857', 'S2A_MSIL1C_20190719T141051_N0208_R110_T21LWK_20190719T155108', 'S2B_MSIL1C_20190919T140049_N0208_R067_T21LWK_20190919T153847', 'S2A_MSIL1C_20180830T140051_N0206_R067